RNN的缺陷，当传递了两条词条后，前面的词条几乎失去了它的作用。在循环网络中，当遍历每个句子时，权重会衰减的很快。  
长短期记忆(long short-term memory, LSTM)网络使用一种特殊的神经网络单元，称为门控循环单元(gated recurrent unit, GRU)。门控循环单元可以有效的保持长、短期记忆，使LSTM能够更精确的处理长句子或文档。

# 长短期记忆(LSTM)
LSTM对循环网络的每一层都引入了状态(state)的概念，而状态作为网络的记忆(memory)。  
在LSTM中，训练神经网络就是管理存储在状态中信息的规则，可以通过训练来学习要记住什么。  
当信息流入LSTM中时，需要经过三个门：遗忘门、输入/候选门、输出门。  
这些门中的每一个都由一个前馈网络层和一个激活函数构成，其中前馈网络包含将要学习的一系列权重。

In [1]:
maxlen = 400
batch_size = 32
embedding_dims = 300
epochs = 2

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM

num_neurons = 50

model = Sequential()
model.add(LSTM(num_neurons, return_sequences=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 400, 50)           70200     
_________________________________________________________________
dropout (Dropout)            (None, 400, 50)           0         
_________________________________________________________________
flatten (Flatten)            (None, 20000)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 20001     
Total params: 90,201
Trainable params: 90,201
Non-trainable params: 0
_________________________________________________________________


与上一章相同的50个神经元，但需要训练但参数比上一章但SimpleRNN要多得多。  
简单RNN有以下权重：
1. 300:对应于输入向量但每个元素
2. 1:对应于偏置项
3. 50:对应于前一个时刻的每个神经元的输出  
  
每个神经元共有351个权重： 351*50=17550
  
LSTM有三个门(总共有4个神经元)：
17550*4=70200

## 遗忘门  
根据给定输入，学习要遗忘多少的记忆。  
遗忘门本身只是一个前馈网络，由n个神经元组成，每个神经元的权重个数为m+n+1，所以在本示例中遗忘门有50个神经元，每个神经元有351（300+50+1）个权重。  
遗忘门的输出向量是某种掩码(多孔掩码)，它会遗忘记忆向量的某些元素。当遗忘门输出值接近于1时，对于该时刻，关联元素中更多的记忆知识会被保留，它越接近于0，遗忘的知识就越多。

## 候选门
一个小网络，学习要加入多少记忆。  
候选门内部有两个独立的神经元，需要做两件事：
1. 决定哪些输入向量元素值得记住(类似遗忘门中的掩码)
2. 将记住的输入元素按规定路线放置到正确的记忆"槽"中。  
  
候选门的第一个部分是一个具有sigmoid激活函数的神经元，其目标是学习要更新记忆向量的哪些输入值，这个神经元类似遗忘门中的掩码。  
这个门的第二个值决定使用多大的值来更新记忆，使用一个tanh激活函数，强制输出值在-1和1之间。

## 输出门
通过掩码结构来权衡应该输出什么。

## 随时间反向传播
这个网络也是通过反向传播算法学习的。  
RNN：易受到梯度消失影响，因为在任意给定时刻，导数都是权重的一个决定因素，因此，当结合不同的学习率，往之前时刻(词条)传播时，经过几次迭代后，权重(和学习率)可能会将梯度缩小到0。在反向传播结束时(相当于序列的开始)，对权重的更新会很小。当权重稍大时也会出现类似的问题：梯度爆炸并不与网络增长成比例。  
LSTM：通过记忆状态避免了梯度消失问题，每个门中的神经元都是通过它们输入进的函数的导数来更新的，即那些在前向传递时更新记忆状态的函数。所以在任何给定的时刻，当一般链式法则反向应用于前向传播时，对神经元的更新只依赖于当前时刻和前一个时刻的记忆状态。这样，整个函数的误差在每个时刻都能更接近神经元，即为所谓的误差传播(error carousel)。  

In [ ]:
# 构建模型
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM

num_neurons = 50

model = Sequential()
model.add(LSTM(num_neurons, return_sequence=True, input_shape=(maxlen, embedding_dims)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile('rmsprop', 'binary_crossentropy', metrics=['accuracy'])
model.summary()

## 面对未知词的处理方式
1. 随机以已知词的向量代替
2. 使用UNK词条

## 生成聊天文字
通过2-gram或3-gram作为训练集样本进行训练

In [3]:
# 导入古腾堡计划数据集
from nltk.corpus import gutenberg

gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [5]:
text = ""
for txt in gutenberg.fileids():
    if "shakespeare" in txt:  # 拼接NLTK中古腾堡语料库中的所有莎士比亚戏剧
        text += gutenberg.raw(txt).lower()
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))  # 为索引建立一个字符字典以便在独热编码中使用
indices = dict((i, c) for i, c in enumerate(chars))  # 将一个独热编码解释回字符时，建立一个反向查找字典
"corpus length: {} , total chars: {}".format(len(text), len(chars))

'corpus length: 375542 , total chars: 50'

In [6]:
print(text[:500])

[the tragedie of julius caesar by william shakespeare 1599]


actus primus. scoena prima.

enter flauius, murellus, and certaine commoners ouer the stage.

  flauius. hence: home you idle creatures, get you home:
is this a holiday? what, know you not
(being mechanicall) you ought not walke
vpon a labouring day, without the signe
of your profession? speake, what trade art thou?
  car. why sir, a carpenter

   mur. where is thy leather apron, and thy rule?
what dost thou with thy best apparrell on


接下来，将原始文本分成数据样本，每个样本都有固定都maxlen个字符

In [7]:
# 组装一个训练集
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print("sequences:", len(sentences))

sequences: 125168


我们现在有125168个训练样本和在它们之后的字符，即模型的目标标签。

In [9]:
# 训练样本的独热编码
import numpy as np

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

在数据集中对每个样本的每个字符进行独热编码，并将其存储在列表x中。  
将独热编码的答案存储在列表y中，然后构建模型

In [10]:
# 组装一个基于字符的LSTM模型来生成文本
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                6450      
_________________________________________________________________
activation (Activation)      (None, 50)                0         
Total params: 98,098
Trainable params: 98,098
Non-trainable params: 0
_________________________________________________________________


RMSprop的工作原理是通过使用"该权重最近梯度大小的平均值"来调整学习率以更新各个权重。  

In [ ]:
# 训练一个莎士比亚风格的聊天机器人
epochs = 6
batch_size = 128
model_structure = model.to_json()
with open("shakes_lstm_model.json", "w") as json_file:
    json_file.write(model_structure)
for i in range(5):
    model.fit(x, y, batch_size=batch_size, epochs=epochs)
    model.save_weights("shake_lstm_weights_().h5".format(i + 1))

这里设置每隔6个训练周期保存模型一次，并继续训练。若损失不再减少，就不需要继续训练，即可停止这个过程。  

In [ ]:
import random

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

log函数除以temperature的效果是使概率分布变平(temperature > 1) 或变尖(temperature < 1)。  
因此小于1的temperature(或称为调用参数中的多样性)倾向于试图更严格的重新创建原始文本。而大于1的temperature回产生更多更多样化的结果，但随着分布变平，学习到的模式开始被遗忘，就会回到胡言乱语的状态，多样性越高就会越有趣。  
numpy随机函数multinomial(num_samples, probability_list, size)将从分布中生成num_samples个样本，其可能的结果由probabilities_list描述，它将输出一个长度为size的列表，该列表等于实验运行的次数，在这种情况下，我们只能从概率分布中抽取一次，我们只需要一个样本。  

In [ ]:
# 使用三种多样化等级产生3种类型文档

import sys
start_index = random.randint(0, len(text) - maxlen - 1)
for diversity in [0.2, 0.5, 1.0]:
    print()
    print('----- diversity: ', diversity)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: ', + sentence + '"')
    sys.stdout.write(generated)
    for i in range(400):
        x = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):  # 为训练网络创建一个输入(seed)
            x[0, t, char_indices[char]] = 1.
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        generated += next_char
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

## 文本生成器的增强方法
1. 增加语料库数量并提升质量
2. 增加模型复杂度
3. 实现一个更精细的字符一致性的算法
4. 句子分段
5. 根据需要在语法、拼写和语气上添加过滤器
6. 生成比实际展示给用户更多的一些结果案例
7. 使用在会话上下文中选择的种子文本引导聊天机器人转向有用的主题
8. 在每一轮对话中使用多个不同的种子文本来探索聊天机器人擅长聊什么领域的话题以及用户认为哪些内容会有帮助。  

文本生成的问题：内容不受控  
  
## 其他记忆机制
LSTM是循环神经网络基本概念的一种扩展，也有如GRU这种对门运算进行优化的扩展。  
GRU的性能于LSTM差不多，计算开销也小。

In [13]:
from keras.models import Sequential
from keras.layers import GRU

model = Sequential()
model.add(GRU(num_neurons, return_sequences=True, input_shape=x[0].shape))

## 更深的网络
堆叠LSTM

In [ ]:
# 两层LSTM
from keras.models import Sequential
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(num_neurons, return_sequences=True, input_shape=x[0].shape))
model.add(LSTM(num_neurons_2, return_sequences=True))

假如要正确构建模型，需要在第一层和中间层使用参数return_sequences=True，才能让每个时刻的输出都能作为下一层时刻的输入。